# Exploring and clustering the neighborhoods in Toronto
### Pt.3: The analysis

In [1]:
import pandas as pd
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

/usr/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
df = pd.read_csv('cbn_coord.csv', 'r', delimiter=',')

In [3]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


### I decided to work with only boroughs that contain the word **Toronto** :

In [4]:
df_final = df[df.Borough.str.contains('Toronto')].reset_index(drop=True)

In [5]:
df_final = df_final.drop('PostalCode', axis=1)

In [6]:
df_final.Borough.value_counts()

Downtown Toronto    18
Central Toronto     9 
West Toronto        6 
East Toronto        5 
Name: Borough, dtype: int64

In [7]:
df_final.shape

(38, 4)

In [8]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="coursera-capstone") 
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geo coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geo coordinates of Toronto are 43.653963, -79.387207.


In [9]:

# I will be using the same coordinates for the previous view
tmap = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_final['Latitude'], df_final['Longitude'], df_final['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='cyan',
        fill_opacity=0.7,
        parse_html=False).add_to(tmap)  
    
tmap

#### Let's use the **Foursquare API** to explore the neighborhoods

In [10]:
CLIENT_ID = 'HIDDEN' # your Foursquare ID
CLIENT_SECRET = 'HIDDEN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

### The following function is to explore the selected neighborhoods in Toronto:

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
toronto_venues = getNearbyVenues(names=df_final['Neighborhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [13]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676845,-79.295225,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676845,-79.295225,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676845,-79.295225,Upper Beaches,43.680563,-79.292869,Neighborhood
3,"The Danforth West, Riverdale",43.683262,-79.355120,Dollarama,43.686300,-79.355893,Discount Store
4,"The Danforth West, Riverdale",43.683262,-79.355120,Sobeys Urban Fresh,43.684690,-79.356350,Grocery Store


In [14]:
toronto_venues.shape # 1732 venues totally

(1732, 7)

### How many venues were returned for each neighborhood

In [15]:
toronto_venues.groupby('Neighborhood')[['Venue']].count().sort_values(by=['Venue'], ascending=False).reset_index()

,Neighborhood,Venue
0,"Adelaide, King, Richmond",100
1,Central Bay Street,100
2,"Design Exchange, Toronto Dominion Centre",100
3,"Ryerson, Garden District",100
4,St. James Town,100
5,"Commerce Court, Victoria Hotel",100
6,Stn A PO Boxes 25 The Esplanade,100
7,Business Reply Mail Processing Centre 969 Eastern,100
8,"First Canadian Place, Underground city",100
9,"Chinatown, Grange Park, Kensington Market",93


####  How many unique categories can be curated from all the returned venues

In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 214 uniques categories.


### Let's analyze each neighborhood separately

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [18]:
toronto_onehot.shape

(1732, 214)

### Now let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.00,0.000000,0.030000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.020000,0.000000,0.000000,0.0,0.030000,0.010000,0.000000,0.000000,0.080000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.00,0.01,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.030000,0.01,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.030000,0.000000,0.000000,0.01,0.000000,0.01,0.000000,0.010000,0.000000,0.020000,0.010000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.050000,0.000000,0.010000,0.000000,0.00

In [20]:
#The new size:
toronto_grouped.shape

(37, 214)

### Neighborhoods along with the top 5 most common venues:

In [21]:
num_top_venues = 5
for neigh in toronto_grouped['Neighborhood']:
    print("----"+neigh+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
          venue  freq
0  Café          0.08
1  Coffee Shop   0.07
2  Hotel         0.05
3  Restaurant    0.03
4  Burger Joint  0.03


----Berczy Park----
          venue  freq
0  Coffee Shop   0.11
1  Restaurant    0.05
2  Cocktail Bar  0.05
3  Lounge        0.03
4  Steakhouse    0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0  Coffee Shop             0.09
1  Restaurant              0.06
2  Café                    0.06
3  Furniture / Home Store  0.06
4  Sandwich Place          0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.07
1  Café         0.05
2  Steakhouse   0.04
3  Hotel        0.04
4  Bar          0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0  Coffee Shop         0.11
1  Italian Restaurant  0.08
2  Pub                 0.05
3  Restaurant      

### Let's put the last received results into a dataframe and display top 10 venues for each neighborhood

In [22]:
def return_most_common_venues(row, num_top_venues):
    '''
    A function to sort the venues in descending order.
    '''
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Hotel,Bakery,Burger Joint,Bar,Asian Restaurant,Steakhouse,Japanese Restaurant,Restaurant
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Bakery,Hotel,Seafood Restaurant,Café,Breakfast Spot,Lounge
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Furniture / Home Store,Restaurant,Café,Sandwich Place,Hotel,Supermarket,Bar,Bakery,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Café,Bar,Steakhouse,Hotel,Pub,American Restaurant,Japanese Restaurant,Italian Restaurant,Pizza Place
4,"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",Coffee Shop,Italian Restaurant,Pub,Restaurant,Gym / Fitness Center,Café,Speakeasy,Park,Sandwich Place,Diner


### Clustering neighborhoods

In [24]:
#Preparing the data: drop non-numeric (nominal) data:
clust = toronto_grouped.drop('Neighborhood', axis=1)

In [25]:
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clust)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 2, 3, 1,
       0, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 0], dtype=int32)

In [26]:
labels = kmeans.labels_
centers = kmeans.cluster_centers_

### Merging the initial and resulting tables on *Neighborhood* column in order to get coordinates:

In [27]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', labels)

In [28]:
merged = df_final

In [29]:
merged = merged.sort_values(by=['Neighborhood']).reset_index(drop=True)
neighborhoods_venues_sorted = neighborhoods_venues_sorted.sort_values(by=['Neighborhood']).reset_index(drop=True)

In [30]:
merged = merged.merge(neighborhoods_venues_sorted, on='Neighborhood')

In [31]:
merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Adelaide, King, Richmond",43.649515,-79.382503,3,Café,Coffee Shop,Hotel,Bakery,Burger Joint,Bar,Asian Restaurant,Steakhouse,Japanese Restaurant,Restaurant
1,Downtown Toronto,Berczy Park,43.645160,-79.373675,3,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Bakery,Hotel,Seafood Restaurant,Café,Breakfast Spot,Lounge
2,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.639410,-79.424362,3,Coffee Shop,Furniture / Home Store,Restaurant,Café,Sandwich Place,Hotel,Supermarket,Bar,Bakery,Italian Restaurant
3,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.648690,-79.385440,3,Coffee Shop,Café,Bar,Steakhouse,Hotel,Pub,American Restaurant,Japanese Restaurant,Italian Restaurant,Pizza Place
4,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",43.640815,-79.399538,3,Coffee Shop,Italian Restaurant,Pub,Restaurant,Gym / Fitness Center,Café,Speakeasy,Park,Sandwich Place,Diner


### Visualizing the clusters:

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

clrs = ['red', 'brown', 'green', 'violet', 'orange', 'blue']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color= clrs[cluster-1],
        fill=True,
        fill_color= 'cyan', # clrs[cluster-1],
        fill_opacity=0.4).add_to(map_clusters)
       
map_clusters

#### Increasing the number of clusters (up to 10) does not affect the rather homogenous structure in the center of the conglomerate, increasing the number of "outliers" on the edges. 

#### Quick look at the data below shows that the most numerous 3rd ('central') cluster consists mostly of public eating places, while the marginal clusters are mostly poblic accomodations (playgrounds, parks, transport lanes, stores etc.). Further analysis requires more accurate classification according to possible research questions.

In [33]:
merged[merged['Cluster Labels'] == 0]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,Lawrence Park,43.728160,-79.387085,0,Construction & Landscaping,Bus Line,Swim School,Wings Joint,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
25,Central Toronto,North Toronto West,43.714523,-79.406960,0,Playground,Gym Pool,Park,Garden,Wings Joint,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
27,Downtown Toronto,Rosedale,43.681940,-79.378474,0,Playground,Park,Bank,Tennis Court,Building,Deli / Bodega,Cupcake Shop,Flower Shop,Fish Market,Fish & Chips Shop
34,East Toronto,The Beaches,43.676845,-79.295225,0,Health Food Store,Pub,Wings Joint,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
36,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120,0,Bus Line,Park,Grocery Store,Discount Store,Wings Joint,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [34]:
merged[merged['Cluster Labels'] == 1]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,"Forest Hill North, Forest Hill West",43.694785,-79.414405,1,Park,Wings Joint,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
21,West Toronto,"High Park, The Junction South",43.659975,-79.462874,1,Convenience Store,Park,Wings Joint,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [35]:
merged[merged['Cluster Labels'] == 2]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.62347,-79.391507,2,Harbor / Marina,Park,Athletics & Sports,Music Venue,Wings Joint,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [36]:
merged[merged['Cluster Labels'] == 3]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Adelaide, King, Richmond",43.649515,-79.382503,3,Café,Coffee Shop,Hotel,Bakery,Burger Joint,Bar,Asian Restaurant,Steakhouse,Japanese Restaurant,Restaurant
1,Downtown Toronto,Berczy Park,43.645160,-79.373675,3,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Bakery,Hotel,Seafood Restaurant,Café,Breakfast Spot,Lounge
2,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.639410,-79.424362,3,Coffee Shop,Furniture / Home Store,Restaurant,Café,Sandwich Place,Hotel,Supermarket,Bar,Bakery,Italian Restaurant
3,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.648690,-79.385440,3,Coffee Shop,Café,Bar,Steakhouse,Hotel,Pub,American Restaurant,Japanese Restaurant,Italian Restaurant,Pizza Place
4,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",43.640815,-79.399538,3,Coffee Shop,Italian Restaurant,Pub,Restaurant,Gym / Fitness Center,Café,Speakeasy,Park,Sandwich Place,Diner
5,Downtown Toronto,"Cabbagetown, St. James Town",43.668160,-79.366602,3,Coffee Shop,Market,Pizza Place,Bakery,Café,Restaurant,Italian Restaurant,Chinese Restaurant,Jewelry Store,Breakfast Spot
6,Downtown Toronto,Central Bay Street,43.656091,-79.384930,3,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Tea Room,Sushi Restaurant,Cosmetics Shop,Bookstore,Plaza,Fast Food Restaurant,Italian Restaurant
7,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653570,-79.397249,3,Café,Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Mexican Restaurant,Dumpling Restaurant,Coffee Shop,Bakery,Chinese Restaurant,Caribbean Restaurant
8,Downtown Toronto,Christie,43.668781,-79.420710,3,Café,Grocery Store,Italian Restaurant,Baby Store,Athletics & Sports,Coffee Shop,Playground,Fast Food Restaurant,Farmers Market,Fish & Chips Shop
9,Downtown Toronto,Church and Wellesley,43.666585,-79.381302,3,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Bubble Tea Shop,Pub,Nightclub,Dance Studio,Men's Store
